##### Sentence Transformers

Sentence Transformers compute vector representations for sentences and paragraphs. The model are based on DistilBERT and perform state-of-the-art results in various task. Text is embedding in vector space such that similar text is close and can efficiently be found using cosine similarity.

We can find sentence transformers distilbert base-nli-stsb-mean-tokens model easily in transformers python library of Hugging Face

! pip install transformers



In [2]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

In [ ]:
! pip install sentence-transformers

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range=(1,1)
stop_words="english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates=count.get_feature_names()

d:\Code\DeepLearning\KeywordExtractor\vKeyword\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [52]:
print(doc_embedding.shape)
print(candidate_embeddings.shape)

(1, 768)
(50, 768)


In [55]:
# Cosine Similarity

from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [56]:
keywords

['class', 'training', 'supervised', 'algorithm', 'learning']

In [4]:
# USing transformers

from transformers import AutoModel, AutoTokenizer

# Defining the model repo
model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens" 

# Download Pytorch Model
model= AutoModel.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

# Transform input tokens
inputs = tokenizer("Hello World !",return_tensors='pt')

# Model output
output=model(**inputs)

In [54]:
doc_token=tokenizer([doc],return_tensors='pt',padding=True,truncation=True)
doc_embedding=model(**doc_token)

cand_token=tokenizer(candidates,return_tensors='pt',padding=True,truncation=True)
cand_embedding=model(**cand_token)

TypeError: forward() got an unexpected keyword argument 'input_ids'

In [53]:
X_doc=doc_embedding['last_hidden_state'].detach().numpy().transpose(1,2,0)
X_cand=cand_embedding['last_hidden_state'].detach().numpy().transpose(1,2,0)
print(X_doc.reshape(-1,X_doc.shape[1]).shape)
print(X_cand.reshape(-1,X_cand.shape[1]).shape)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [43]:
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(X_doc.reshape(-1,X_doc.shape[0]), X_cand.reshape(-1,X_cand.shape[0]))
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 156 while Y.shape[1] == 5